In [8]:
##############################################################################
## 공통 설정 및 라이브러리 임포트
##############################################################################
import os
import re
from typing import List, TypedDict

from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.tools import tool
from langchain_community.document_loaders import TextLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import (AIMessage, BaseMessage, HumanMessage,
                                      ToolMessage)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import END, StateGraph

##############################################################################
## 문제 4-1: 카페 메뉴 도구(Tool) 호출 체인 구현 (LangChain 사용)
##############################################################################

print("="*30)
print("문제 4-1: LangChain Tool Calling 시작")
print("="*30)

# --- 1. 환경변수 설정 및 벡터 DB 구축 ---

# .env 파일에서 API 키를 로드합니다.
# 이 파일은 프로젝트 최상위 폴더에 위치해야 합니다.
load_dotenv()

# [!!] 만약 .env 파일이 계속 문제를 일으킨다면, 아래 코드의 주석을 풀고 직접 키를 입력하세요.
# os.environ['OPENAI_API_KEY'] = "sk-여기에_실제_API_키를_입력하세요"
# os.environ['TAVILY_API_KEY'] = "tvly-여기에_실제_API_키를_입력하세요"


# 텍스트 파일 로드 및 분할 (경로 수정!)
# '과제' 폴더에서 한 단계 위로 올라가 'data' 폴더로 접근
loader = TextLoader('../data/cafe_menu.txt', encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 임베딩 모델 로드 및 벡터 DB 생성/저장
embeddings = OpenAIEmbeddings()
# db 저장 경로는 노트북 파일 기준이므로, 프로젝트 루트에 저장되도록 경로 수정
db_path = '../../db/cafe_db'
db = FAISS.from_documents(docs, embeddings)
db.save_local(db_path)
print(f"벡터 DB가 '{db_path}' 폴더에 성공적으로 저장되었습니다.")


# --- 2. 도구 정의 및 LLM 바인딩 ---

# a) Tavily 웹 검색 도구
tavily_search_func = TavilySearchResults(max_results=1, name="tavily_search")

# b) 위키피디아 요약 도구
@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 정보를 검색하고 그 결과를 요약합니다. 커피 역사, 음료 제조법 등 일반적인 지식에 사용됩니다."""
    wikipedia = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
    return wikipedia.run(query)

# c) 로컬 카페 메뉴 DB 검색 도구
@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 데이터베이스에서 메뉴의 가격, 재료, 설명 등의 정보를 검색합니다."""
    vector_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    retriever = vector_db.as_retriever(search_kwargs={"k": 1})
    docs = retriever.get_relevant_documents(query)
    return docs[0].page_content if docs else "관련 메뉴 정보를 찾을 수 없습니다."

# 도구 리스트 정의
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

# LLM에 도구 바인딩
llm = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)
print("3개의 도구가 LLM에 성공적으로 바인딩되었습니다.")


# --- 3. 도구 호출 체인 구현 및 테스트 ---

def run_tool_chain(user_input: str):
    """사용자 질문에 따라 적절한 도구를 호출하는 체인"""
    response = llm.invoke([HumanMessage(content=user_input)])
    if not response.tool_calls:
        return response.content
    tool_outputs = []
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"]
        print(f"Tool Calling: '{tool_name}' with args: {tool_call['args']}")
        if tool_name == "tavily_search":
            output = tavily_search_func.invoke(tool_call["args"]["query"])
        elif tool_name == "wiki_summary":
            output = wiki_summary.invoke(tool_call["args"]["query"])
        elif tool_name == "db_search_cafe_func":
            output = db_search_cafe_func.invoke(tool_call["args"]["query"])
        else:
            raise ValueError(f"Unknown tool: {tool_name}")
        tool_outputs.append(ToolMessage(content=str(output), tool_call_id=tool_call["id"]))
    final_response = llm.invoke([HumanMessage(content=user_input), response] + tool_outputs)
    return final_response.content

question = "아메리카노의 가격과 특징은 무엇인가요?"
answer = run_tool_chain(question)
print(f"\n[질문]: {question}")
print(f"[답변]: {answer}")
print("\n문제 4-1 실행 완료\n")


##############################################################################
## 문제 4-2: 조건부 분기가 있는 메뉴 추천 시스템 (LangGraph 사용)
##############################################################################

print("="*30)
print("문제 4-2: LangGraph 메뉴 추천 시스템 시작")
print("="*30)

# --- 1. 상태 정의 및 노드(기능) 구현 ---

class CafeState(TypedDict):
    messages: List[BaseMessage]

def extract_menu_info(doc_content: str) -> dict:
    menu_name_match = re.search(r'\d+\.\s*(.+)', doc_content)
    menu_name = menu_name_match.group(1).strip() if menu_name_match else '이름 정보 없음'
    price_match = re.search(r'가격:\s*(.+)', doc_content)
    price = price_match.group(1).strip() if price_match else "가격 정보 없음"
    description_match = re.search(r'설명:\s*(.+)', doc_content, re.DOTALL)
    description = description_match.group(1).strip() if description_match else "설명 없음"
    return {"name": menu_name, "price": price, "description": description}

menu_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
print(f"'{db_path}'의 벡터 DB를 성공적으로 재사용합니다.")

def classify_question_node(state: CafeState):
    user_message = state["messages"][-1].content.lower()
    if "가격" in user_message or "얼마" in user_message:
        return "price_inquiry"
    elif "추천" in user_message or "어떤" in user_message or "마실" in user_message:
        return "recommendation_request"
    else:
        return "menu_inquiry"

def price_info_node(state: CafeState):
    docs = menu_db.similarity_search("모든 메뉴의 가격", k=10)
    response_text = "카페 메뉴 전체 가격 정보입니다:\n\n"
    for doc in docs:
        info = extract_menu_info(doc.page_content)
        response_text += f"- {info['name']}: {info['price']}\n"
    return {"messages": [AIMessage(content=response_text)]}

def recommend_menu_node(state: CafeState):
    user_message = state["messages"][-1].content
    docs = menu_db.similarity_search(user_message, k=3)
    if not docs:
        docs = menu_db.similarity_search("인기 메뉴", k=3)
    response_text = "이런 메뉴는 어떠신가요? ☕\n\n"
    for doc in docs:
        info = extract_menu_info(doc.page_content)
        response_text += f"**{info['name']}**: {info['description']}\n"
    return {"messages": [AIMessage(content=response_text)]}

def menu_info_node(state: CafeState):
    user_message = state["messages"][-1].content
    docs = menu_db.similarity_search(user_message, k=1)
    if docs:
        info = extract_menu_info(docs[0].page_content)
        response_text = f"**{info['name']}**\n- **가격**: {info['price']}\n- **설명**: {info['description']}"
    else:
        response_text = "죄송합니다, 요청하신 메뉴 정보를 찾을 수 없습니다."
    return {"messages": [AIMessage(content=response_text)]}

# --- 2. 그래프 생성 및 실행 ---

graph_builder = StateGraph(CafeState)
graph_builder.add_node("price_inquiry", price_info_node)
graph_builder.add_node("recommendation_request", recommend_menu_node)
graph_builder.add_node("menu_inquiry", menu_info_node)
graph_builder.add_conditional_edges(
    "__start__",
    classify_question_node,
    {
        "price_inquiry": "price_inquiry",
        "recommendation_request": "recommendation_request",
        "menu_inquiry": "menu_inquiry",
    }
)
graph_builder.add_edge("price_inquiry", END)
graph_builder.add_edge("recommendation_request", END)
graph_builder.add_edge("menu_inquiry", END)
app = graph_builder.compile()
print("LangGraph 애플리케이션이 성공적으로 컴파일되었습니다.")

test_questions = [
    "자몽 허니 블랙티에 대해 알려줘",
    "메뉴 전체 가격이 어떻게 되나요?",
    "달달하고 시원한 음료 추천해 줄래?",
    "카푸치노는 얼마야?"
]

for q in test_questions:
    print(f"\n--- 질문: {q} ---")
    result = app.invoke({"messages": [HumanMessage(content=q)]})
    print(result['messages'][-1].content)
    
print("\n문제 4-2 실행 완료")
print("="*30)

문제 4-1: LangChain Tool Calling 시작


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: gsk_xTuE********************************************hIFP. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}